## The model 6 about Metrics for Classification model and Multiclassification

## homework by matrix error and metrics for classification

In [119]:
import pandas as pd

In [120]:
titanic = pd.read_csv('train.csv')
print('\nForm:', titanic.shape)
print('\nName_features:', titanic.columns)
print('\nType:', titanic.dtypes)
titanic.head(10)


Form: (891, 12)

Name_features: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Type: PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Check data for NULL values

In [121]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [122]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [123]:
# fill all fills with none in column age
import numpy as np

mean_age_sex = (titanic
                .groupby('Sex')
                .agg({'Age': 'mean'}))

mean_age_condition = np.where(titanic['Age'].isnull() == True,
                              mean_age_sex.loc[titanic['Sex'], 'Age'],
                              titanic['Age'])

titanic['Age'] = mean_age_condition

In [124]:
# fill all with none columns cabin embarked
titanic.fillna('U', inplace=True)

In [125]:
titanic.iloc[5:20].Age

5     30.726645
6     54.000000
7      2.000000
8     27.000000
9     14.000000
10     4.000000
11    58.000000
12    20.000000
13    39.000000
14    14.000000
15    55.000000
16     2.000000
17    30.726645
18    31.000000
19    27.915709
Name: Age, dtype: float64

In [126]:
titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [129]:
titanic['Sex'] = titanic['Sex'].replace(['male', 'female'], value=[0, 1])
titanic['Embarked'] = titanic['Embarked'].replace(["U", "S", "C", "Q"], value=[0, 1, 2, 3])

In [135]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1


In [143]:
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [138]:
y = titanic['Survived']
titanic.drop(['Survived'], axis=1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.000000,1,0,7.2500,1
1,1,1,38.000000,1,0,71.2833,2
2,3,1,26.000000,0,0,7.9250,1
3,1,1,35.000000,1,0,53.1000,1
4,3,0,35.000000,0,0,8.0500,1
...,...,...,...,...,...,...,...
886,2,0,27.000000,0,0,13.0000,1
887,1,1,19.000000,0,0,30.0000,1
888,3,1,27.915709,1,2,23.4500,1
889,1,0,26.000000,0,0,30.0000,2


In [140]:
x_train, x_test, y_train, y_test = train_test_split(titanic, y, random_state=17)

In [142]:
x_train.shape, x_test.shape

((668, 8), (223, 8))

In [145]:
y_train.value_counts()

0    418
1    250
Name: Survived, dtype: int64